# **Feature Engineering & Data Preparation**

## Objectives

* Prepare the `cleaned_data.csv` dataset for machine learning (ML), following the modelling pipeline described in the project brief.
* Engineer the features identified during EDA as relevant predictors of `SalePrice` (e.g., GrLivArea, TotalBsmtSF, LotArea, GarageArea, OverallQual, and categorical quality ratings).
* Convert all categorical variables into numerical format using suitable encoding techniques.
* Apply data transformations informed by EDA - for example:
  * log-transform `SalePrice` if required
  * handle skewed numerical features
  * normalise or standardise features if necessary
* Create the final modelling dataset (`X` and `y`) and perform a train–test split to support model evaluation.
* Apply identical feature engineering steps to `inherited_houses.csv` so predictions can be generated consistently later in the dashboard.

## Inputs

* `data/processed/cleaned_data.csv`
  * Cleaned and fully validated Ames dataset produced in 02_data_cleaning.ipynb.
* `data/raw/inherited_houses.csv`
  * Client’s four inherited homes, used for aligning preprocessing and generating predictions later.
* Insights from 03_exploratory_analysis.ipynb:
  * Most predictive numerical features
  * Categorical variables requiring encoding
  * Level of skewness in key features
  * Confirmation that inherited houses fall within normal market ranges
* Project requirements and modelling expectations from the Project Plan 
  * Final model must achieve **R² ≥ 0.75**
  * Dataset must be prepared following a clear ML pipeline
  * Model will later be used in a Streamlit dashboard with prediction capability

## Outputs

* A fully engineered modelling dataset including:
  * Encoded categorical variables
  * Transformed numerical variables (if needed)
  * Final feature matrix `X` and target `y`
  * `X_train`, `X_test`, `y_train`, `y_test` split for modelling
* A processed version of the inherited dataset with exactly the same encoding and transformations applied.
* A list of final selected modelling features.
* Saved processed datasets in:
  * `data/processed/engineered_training_data.csv`
  * `data/processed/engineered_inherited_houses.csv`
* Clear justification for all feature engineering decisions (based on EDA).

## Additional Comments

* All feature engineering decisions must directly support the modelling requirements in the project specification and dashboard design (prediction page, feature insights page, etc.).
* No new external features will be added; only the original cleaned dataset will be transformed.
* This notebook ensures that the dataset used in Notebook 05 (Model Training) is clean, fully numeric, consistent, and suitable for supervised regression modelling.
* The engineered feature set will later support:
  * Model performance evaluation (R² score)
  * Price predictions for the 4 inherited houses
  * Real-time predictions in the Streamlit dashboard
* Every transformation applied here must be reproducible and applied identically during inference (prediction).


---

# Briefly inspect the cleaned data

### Step 1: Load cleaned datasets for feature engineering

In [1]:
import pandas as pd

# Load the cleaned Ames housing dataset
housing_df = pd.read_csv("data/processed/cleaned_data.csv")

# Load the inherited houses dataset
inherited_df = pd.read_csv("data/raw/inherited_houses.csv")

# Quick checks
housing_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,548,RFn,2003.0,...,8450,65.0,196.0,61,5,7,856,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,460,RFn,1976.0,...,9600,80.0,0.0,0,8,6,1262,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,608,RFn,2001.0,...,11250,68.0,162.0,42,5,7,920,2001,2002,223500
3,961,0.0,3.0,No,216,ALQ,540,642,Unf,1998.0,...,9550,60.0,0.0,35,5,7,756,1915,1970,140000
4,1145,0.0,4.0,Av,655,GLQ,490,836,RFn,2000.0,...,14260,84.0,350.0,84,5,8,1145,2000,2000,250000


In this step, I load the `cleaned_data.csv` dataset, along with the client’s inherited properties (`inherited_houses.csv`). 

The `housing_df` DataFrame will be used to engineer the features needed for modelling the target variable `SalePrice`, while `inherited_df` will be kept in sync with the same transformations. This ensures that any encoding, scaling, or transformations applied during feature engineering can later be applied consistently when generating price predictions for the four inherited houses.

---

# Define Features (X) and Target (y)

In [2]:
# Step 2: Define Features (X) and Target (y)

# The target variable we want to predict
y = housing_df["SalePrice"]

# Features used for modelling (excluding SalePrice)
# Selected based on EDA findings and project business requirements
selected_features = [
    "GrLivArea",        # Above-ground living area
    "LotArea",          # Lot size
    "TotalBsmtSF",      # Basement size
    "GarageArea",       # Garage size
    "OverallQual",      # Overall material and finish quality
    "OverallCond",      # Overall condition rating
    "KitchenQual",      # Categorical: kitchen quality
    "BsmtExposure",     # Categorical: basement exposure
    "BsmtFinType1",     # Categorical: basement finish type
    "GarageFinish"      # Categorical: garage finish quality
]

# Create the feature matrix
X = housing_df[selected_features]

X.head()

,GrLivArea,LotArea,TotalBsmtSF,GarageArea,OverallQual,OverallCond,KitchenQual,BsmtExposure,BsmtFinType1,GarageFinish
0,1710,8450,856,548,7,5,Gd,No,GLQ,RFn
1,1262,9600,1262,460,6,8,TA,Gd,ALQ,RFn
2,1786,11250,920,608,7,5,Gd,Mn,GLQ,RFn
3,1717,9550,756,642,7,5,Gd,No,ALQ,Unf
4,2198,14260,1145,836,8,5,Gd,Av,GLQ,RFn


### Step 2: Inspecting the selected feature matrix (X)

The table above shows the first five rows of the feature matrix `X` after selecting the variables that will be used for modelling.

The columns include a mix of numerical and categorical predictors that our earlier EDA identified as important for explaining variation in `SalePrice`:

- **GrLivArea, LotArea, TotalBsmtSF, GarageArea** – continuous measures of size (living area, land, basement, and garage).
- **OverallQual, OverallCond** – numeric quality and condition ratings recorded on an ordinal scale (higher values = better quality/condition).
- **KitchenQual, BsmtExposure, BsmtFinType1, GarageFinish** – categorical descriptors of kitchen quality, basement exposure, basement finish type, and garage finish.

From this preview we can see that:

- All selected columns are present and correctly loaded from `housing_df`.
- The numerical features contain realistic values consistent with the ranges observed during EDA.
- The categorical features use the expected Ames coding scheme (e.g. `Gd`, `TA`, `GLQ`, `RFn`, `Unf`).

This confirms that the chosen predictors are correctly assembled into `X` and ready for the next steps of feature engineering, where the categorical variables will be encoded and numerical features transformed as needed for modelling.


--- 

# Numerical and Categorical variables

In [3]:
# Step 3: Identify numerical and categorical variables

# Numerical features (int or float)
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Categorical features (object/string)
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

numerical_features, categorical_features

(['GrLivArea',
  'LotArea',
  'TotalBsmtSF',
  'GarageArea',
  'OverallQual',
  'OverallCond'],
 ['KitchenQual', 'BsmtExposure', 'BsmtFinType1', 'GarageFinish'])

### Step 3: Identify numerical and categorical variables

In this step, I identified which features are numerical and which are categorical within the selected feature set `X`.

From the selected modelling features:

**Numerical features:**  
- GrLivArea  
- LotArea  
- TotalBsmtSF  
- GarageArea  
- OverallQual  
- OverallCond  

**Categorical features:**  
- KitchenQual  
- BsmtExposure  
- BsmtFinType1  
- GarageFinish  

This classification will guide the next steps of feature engineering, where I apply appropriate encoding and transformations to prepare the dataset for modelling.


##### Choosing the appropriate encoding method for categorical variables

Before encoding the categorical features, it is important to decide whether each variable should use ordinal encoding  or one-hot encoding. This decision depends on whether the categories represent a meaningful order or are simply different labels with no ranking.

In this project, all four selected categorical variables represent **quality levels** or **finish types**, meaning they naturally follow an ordered hierarchy. Because of this, using **ordinal encoding** is the most appropriate approach. 

**Variables and their justification:**

- **KitchenQual** — Represents kitchen quality (Ex > Gd > TA > Fa).  
  Since these categories reflect increasing levels of quality, ordinal encoding preserves this order.

- **BsmtExposure** — Indicates the level of basement exposure (Gd > Av > Mn > No).  
  These categories form a clear progression from high exposure to none, making them ordinal.

- **BsmtFinType1** — Basement finish type (GLQ > ALQ > BLQ > Rec > LwQ > Unf).  
  These categories represent increasing levels of finish quality, suitable for ordinal encoding.

- **GarageFinish** — Garage finish level (Fin > RFn > Unf).  
  These categories follow a logical quality order and should therefore be ordinal.